In [1]:
import tensorflow as tf
import keras
from keras import layers
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
!pip install tensorflow-model-optimization
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.sparsity import keras as sparsity
from tensorflow.keras.models import load_model
from google.colab import files
import shutil
import gc
from keras.datasets import cifar10
from tensorflow.keras import layers
import shutil
from google.colab import drive
drive.mount('/content/drive')

np.random.seed(42)
tf.random.set_seed(42)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 4.5 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
num_classes = 10
input_shape = (32, 32, 3)

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



170498071/170498071 [==============================] - 4s 0us/step
x_train shape: (50000, 32, 32, 3, 1)
50000 train samples
10000 test samples


In [3]:
model_name = 'resnet'
dataset = 'CIFAR10'

---

# Distillation

In [4]:
def train_custom(model, train_data, train_labels, test_data, test_labels):
    initailsoftmax = model.student.predict(test_data)
    model.student.save_weights(init_weights)
    model.student.save(init_model)
    initaildf = pd.DataFrame(initailsoftmax)
    filename = filepath + '0_softmax.csv'
    initaildf.to_csv(filename,index=False)
    epoch = 1
    while epoch < 26:
        print(f"Epoch {epoch}:")

        # Training on one epoch
        model.fit(train_data, train_labels, epochs=1, batch_size=64, verbose=1)

        # Evaluate on the test dataset
        results = model.student.predict(test_data)
        softmax_df = pd.DataFrame(results)
        filename = filepath+str(epoch)+'_softmax.csv'
        softmax_df.to_csv(filename,index=False)
        loss,accuracy = model.student.evaluate(test_data,test_labels)
        print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

        if epoch ==25:
          model.student.save_weights(final_weights)
          model.student.save(final_model)
        epoch += 1



In [5]:
class Distiller(tf.keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha,
        temperature,
    ):
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)

            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )

        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)
        print(y_prediction)
        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

In [6]:
teacher_model_path = '/content/base/resnetCIFAR10_final.tf'
techer_model = load_model(teacher_model_path)

# Student 0.1 Alpha

In [7]:
np.random.seed(42)
tf.random.set_seed(42)
model_path = '/content/base/resnetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=3,
)
filepath = '/content/student_same/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 7s 12ms/step
Epoch 1:
313/313 [==============================] - 5s 11ms/step - loss: 1.6924 - accuracy: 0.3854
Test Loss: 1.6924479007720947, Test Accuracy: 0.3853999972343445
Epoch 2:
313/313 [==============================] - 4s 13ms/step - loss: 1.4757 - accuracy: 0.4654
Test Loss: 1.475663423538208, Test Accuracy: 0.46540001034736633
Epoch 3:
313/313 [==============================] - 4s 11ms/step - loss: 1.4602 - accuracy: 0.4881
Test Loss: 1.4602166414260864, Test Accuracy: 0.48809999227523804
Epoch 4:
313/313 [==============================] - 4s 14ms/step - loss: 1.6003 - accuracy: 0.4525
Test Loss: 1.600266456604004, Test Accuracy: 0.45249998569488525
Epoch 5:
313/313 [==============================] - 4s 13ms/step - loss: 1.3973 - accuracy: 0.5230
Test Loss: 1.3972538709640503, Test Accuracy: 0.5230000019073486
Epoch 6:
313/313 [==============================] - 3s 11ms/step - loss: 1.3811 - accuracy: 0.5528
Test Loss: 1.38105857372

In [8]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same", 'zip', "student_same")
colab_link = "/content/student_same.zip"
gdrive_link = "/content/drive/MyDrive/ResNet_D_C10"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/ResNet_D_C10/student_same.zip'

# Student 0.1 (2)

In [9]:
np.random.seed(24)
tf.random.set_seed(24)
model_path = '/content/base/resnetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=3,
)
filepath = '/content/student_same_2/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_2/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 4s 10ms/step
Epoch 1:
313/313 [==============================] - 6s 14ms/step - loss: 1.6164 - accuracy: 0.4116
Test Loss: 1.6164369583129883, Test Accuracy: 0.4115999937057495
Epoch 2:
313/313 [==============================] - 4s 12ms/step - loss: 1.6324 - accuracy: 0.4271
Test Loss: 1.6324255466461182, Test Accuracy: 0.4271000027656555
Epoch 3:
313/313 [==============================] - 3s 11ms/step - loss: 1.3878 - accuracy: 0.5106
Test Loss: 1.3877919912338257, Test Accuracy: 0.5105999708175659
Epoch 4:
313/313 [==============================] - 3s 11ms/step - loss: 1.5060 - accuracy: 0.4923
Test Loss: 1.5059617757797241, Test Accuracy: 0.49230000376701355
Epoch 5:
313/313 [==============================] - 4s 13ms/step - loss: 1.4672 - accuracy: 0.5080
Test Loss: 1.4671907424926758, Test Accuracy: 0.5080000162124634
Epoch 6:
313/313 [==============================] - 4s 12ms/step - loss: 1.6384 - accuracy: 0.4890
Test Loss: 1.63839042186

In [10]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_2", 'zip', "student_same_2")
colab_link = "/content/student_same_2.zip"
gdrive_link = "/content/drive/MyDrive/ResNet_D_C10"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/ResNet_D_C10/student_same_2.zip'

# Student 0.1 (3)

In [11]:
np.random.seed(24)
tf.random.set_seed(24)
model_path = '/content/base/resnetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=3,
)
filepath = '/content/student_same_3/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_3/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 4s 9ms/step
Epoch 1:
313/313 [==============================] - 5s 12ms/step - loss: 1.6373 - accuracy: 0.4089
Test Loss: 1.6373223066329956, Test Accuracy: 0.4088999927043915
Epoch 2:
313/313 [==============================] - 3s 11ms/step - loss: 1.6529 - accuracy: 0.4244
Test Loss: 1.6529041528701782, Test Accuracy: 0.4244000017642975
Epoch 3:
313/313 [==============================] - 4s 12ms/step - loss: 1.6968 - accuracy: 0.4235
Test Loss: 1.6967726945877075, Test Accuracy: 0.4235000014305115
Epoch 4:
313/313 [==============================] - 4s 12ms/step - loss: 1.6419 - accuracy: 0.4555
Test Loss: 1.6419156789779663, Test Accuracy: 0.4555000066757202
Epoch 5:
313/313 [==============================] - 3s 11ms/step - loss: 2.2289 - accuracy: 0.3663
Test Loss: 2.228872776031494, Test Accuracy: 0.36629998683929443
Epoch 6:
313/313 [==============================] - 4s 14ms/step - loss: 1.7209 - accuracy: 0.4866
Test Loss: 1.7208732366561

In [12]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_2", 'zip', "student_same_2")
colab_link = "/content/student_same_2.zip"
gdrive_link = "/content/drive/MyDrive/ResNet_D_C10"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/ResNet_D_C10/student_same_2.zip'

# Student same Alpha 0.5

In [13]:
np.random.seed(42)
tf.random.set_seed(42)
model_path = '/content/base/resnetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=3,
)
filepath = '/content/student_same_50/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_50/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 5s 9ms/step
Epoch 1:
313/313 [==============================] - 4s 11ms/step - loss: 1.6401 - accuracy: 0.4013
Test Loss: 1.6401171684265137, Test Accuracy: 0.40130001306533813
Epoch 2:
313/313 [==============================] - 3s 11ms/step - loss: 1.5821 - accuracy: 0.4362
Test Loss: 1.58207368850708, Test Accuracy: 0.43619999289512634
Epoch 3:
313/313 [==============================] - 4s 13ms/step - loss: 1.4480 - accuracy: 0.4796
Test Loss: 1.447980523109436, Test Accuracy: 0.4796000123023987
Epoch 4:
313/313 [==============================] - 4s 13ms/step - loss: 1.3335 - accuracy: 0.5297
Test Loss: 1.3335378170013428, Test Accuracy: 0.529699981212616
Epoch 5:
313/313 [==============================] - 4s 13ms/step - loss: 1.6090 - accuracy: 0.4665
Test Loss: 1.6089732646942139, Test Accuracy: 0.46650001406669617
Epoch 6:
313/313 [==============================] - 3s 11ms/step - loss: 1.4195 - accuracy: 0.5264
Test Loss: 1.41950571537017

In [14]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_50", 'zip', "student_same_50")
colab_link = "/content/student_same_50.zip"
gdrive_link = "/content/drive/MyDrive/ResNet_D_C10"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/ResNet_D_C10/student_same_50.zip'

# Student 0.5 (2)

In [15]:
np.random.seed(24)
tf.random.set_seed(24)
model_path = '/content/base/resnetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=3,
)
filepath = '/content/student_same_50_2/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_50_2/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 5s 10ms/step
Epoch 1:
313/313 [==============================] - 5s 12ms/step - loss: 1.6757 - accuracy: 0.3854
Test Loss: 1.6756867170333862, Test Accuracy: 0.3853999972343445
Epoch 2:
313/313 [==============================] - 4s 13ms/step - loss: 1.5200 - accuracy: 0.4531
Test Loss: 1.5199692249298096, Test Accuracy: 0.4530999958515167
Epoch 3:
313/313 [==============================] - 4s 12ms/step - loss: 1.6189 - accuracy: 0.4439
Test Loss: 1.6189426183700562, Test Accuracy: 0.4438999891281128
Epoch 4:
313/313 [==============================] - 4s 11ms/step - loss: 1.6185 - accuracy: 0.4611
Test Loss: 1.6184903383255005, Test Accuracy: 0.4611000120639801
Epoch 5:
313/313 [==============================] - 4s 13ms/step - loss: 1.5186 - accuracy: 0.5146
Test Loss: 1.5186046361923218, Test Accuracy: 0.5145999789237976
Epoch 6:
313/313 [==============================] - 4s 12ms/step - loss: 1.3044 - accuracy: 0.5699
Test Loss: 1.304359555244

In [16]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_50_2", 'zip', "student_same_50_2")
colab_link = "/content/student_same_50_2.zip"
gdrive_link = "/content/drive/MyDrive/ResNet_D_C10"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/ResNet_D_C10/student_same_50_2.zip'

# Student 0.5 (3)

In [17]:
np.random.seed(2)
tf.random.set_seed(2)
model_path = '/content/base/resnetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=3,
)
filepath = '/content/student_same_50_3/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_50_3/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 4s 9ms/step
Epoch 1:
313/313 [==============================] - 5s 11ms/step - loss: 1.6252 - accuracy: 0.4135
Test Loss: 1.625162959098816, Test Accuracy: 0.41350001096725464
Epoch 2:
313/313 [==============================] - 4s 14ms/step - loss: 1.4506 - accuracy: 0.4729
Test Loss: 1.450607419013977, Test Accuracy: 0.47290000319480896
Epoch 3:
313/313 [==============================] - 4s 14ms/step - loss: 1.6693 - accuracy: 0.4283
Test Loss: 1.6693238019943237, Test Accuracy: 0.42829999327659607
Epoch 4:
313/313 [==============================] - 4s 11ms/step - loss: 1.5023 - accuracy: 0.4950
Test Loss: 1.5023282766342163, Test Accuracy: 0.4950000047683716
Epoch 5:
313/313 [==============================] - 4s 12ms/step - loss: 1.3674 - accuracy: 0.5332
Test Loss: 1.3674193620681763, Test Accuracy: 0.5332000255584717
Epoch 6:
313/313 [==============================] - 4s 11ms/step - loss: 1.2597 - accuracy: 0.5762
Test Loss: 1.259728670120

In [18]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_50_3", 'zip', "student_same_50_3")
colab_link = "/content/student_same_50_3.zip"
gdrive_link = "/content/drive/MyDrive/ResNet_D_C10"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/ResNet_D_C10/student_same_50_3.zip'

# Student same Alpha 0.9

In [9]:
np.random.seed(42)
tf.random.set_seed(42)
model_path = '/content/base/resnetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=3,
)

filepath = '/content/student_same_90/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'

!mkdir '/content/student_same_90/'

train_custom(distiller, x_train,y_train,x_test,y_test)

mkdir: cannot create directory ‘/content/student_same_90/’: File exists
313/313 [==============================] - 4s 9ms/step
Epoch 1:
313/313 [==============================] - 5s 12ms/step - loss: 1.6180 - accuracy: 0.4110
Test Loss: 1.6180131435394287, Test Accuracy: 0.41100001335144043
Epoch 2:
313/313 [==============================] - 4s 12ms/step - loss: 1.5289 - accuracy: 0.4619
Test Loss: 1.5289283990859985, Test Accuracy: 0.461899995803833
Epoch 3:
313/313 [==============================] - 4s 14ms/step - loss: 1.6699 - accuracy: 0.4443
Test Loss: 1.669896125793457, Test Accuracy: 0.44429999589920044
Epoch 4:
313/313 [==============================] - 4s 13ms/step - loss: 1.5143 - accuracy: 0.4700
Test Loss: 1.5142723321914673, Test Accuracy: 0.4699999988079071
Epoch 5:
313/313 [==============================] - 4s 12ms/step - loss: 1.3627 - accuracy: 0.5214
Test Loss: 1.3626971244812012, Test Accuracy: 0.521399974822998
Epoch 6:
313/313 [==============================] - 4s

In [10]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_90", 'zip', "student_same_90")
colab_link = "/content/student_same_90.zip"
gdrive_link = "/content/drive/MyDrive/ResNet_D_C10"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/ResNet_D_C10/student_same_90.zip'

# Student 0.9 (2)

In [11]:
np.random.seed(24)
tf.random.set_seed(24)
model_path = '/content/base/resnetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=3,
)

filepath = '/content/student_same_90_2/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'

!mkdir '/content/student_same_90_2/'

train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 4s 10ms/step
Epoch 1:
313/313 [==============================] - 6s 15ms/step - loss: 1.6169 - accuracy: 0.4052
Test Loss: 1.6169054508209229, Test Accuracy: 0.4052000045776367
Epoch 2:
313/313 [==============================] - 4s 14ms/step - loss: 1.4512 - accuracy: 0.4719
Test Loss: 1.4511817693710327, Test Accuracy: 0.47189998626708984
Epoch 3:
313/313 [==============================] - 4s 12ms/step - loss: 1.5560 - accuracy: 0.4452
Test Loss: 1.5560330152511597, Test Accuracy: 0.44519999623298645
Epoch 4:
313/313 [==============================] - 4s 14ms/step - loss: 1.4375 - accuracy: 0.4933
Test Loss: 1.437536358833313, Test Accuracy: 0.4932999908924103
Epoch 5:
313/313 [==============================] - 4s 13ms/step - loss: 1.5000 - accuracy: 0.5080
Test Loss: 1.4999932050704956, Test Accuracy: 0.5080000162124634
Epoch 6:
313/313 [==============================] - 4s 12ms/step - loss: 1.4635 - accuracy: 0.5392
Test Loss: 1.46346950531

In [12]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_90_2", 'zip', "student_same_90_2")
colab_link = "/content/student_same_90_2.zip"
gdrive_link = "/content/drive/MyDrive/ResNet_D_C10"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/ResNet_D_C10/student_same_90_2.zip'

# Student 0.9 (3)

In [7]:
np.random.seed(2)
tf.random.set_seed(2)
model_path = '/content/base/resnetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=3,
)

filepath = '/content/student_same_90_3/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'

!mkdir '/content/student_same_90_3/'

train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 7s 12ms/step
Epoch 1:
313/313 [==============================] - 5s 11ms/step - loss: 1.6404 - accuracy: 0.4032
Test Loss: 1.6404237747192383, Test Accuracy: 0.4032000005245209
Epoch 2:
313/313 [==============================] - 4s 13ms/step - loss: 1.5052 - accuracy: 0.4606
Test Loss: 1.5052332878112793, Test Accuracy: 0.46059998869895935
Epoch 3:
313/313 [==============================] - 4s 12ms/step - loss: 1.4379 - accuracy: 0.4913
Test Loss: 1.4379280805587769, Test Accuracy: 0.49129998683929443
Epoch 4:
313/313 [==============================] - 4s 14ms/step - loss: 1.3579 - accuracy: 0.5114
Test Loss: 1.357863187789917, Test Accuracy: 0.5113999843597412
Epoch 5:
313/313 [==============================] - 3s 11ms/step - loss: 1.4162 - accuracy: 0.5294
Test Loss: 1.4161546230316162, Test Accuracy: 0.5293999910354614
Epoch 6:
313/313 [==============================] - 4s 11ms/step - loss: 1.3414 - accuracy: 0.5569
Test Loss: 1.34139192104

In [8]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_90_3", 'zip', "student_same_90_3")
colab_link = "/content/student_same_90_3.zip"
gdrive_link = "/content/drive/MyDrive/ResNet_D_C10"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/ResNet_D_C10/student_same_90_3.zip'

---